In [1]:
import warnings
import re

from scripts import train, infer, evaluate
from AVDeepfake1Mpp.code.metadata_generator import generate_metadata
from utils.check_corruption import check_videos_for_corruption

%load_ext autoreload
%autoreload 2

# ignore certain warnings
warnings.filterwarnings(
    "ignore", 
    message="The video decoding and encoding capabilities of torchvision are deprecated from version 0.22"
)

/home/ben/Thesis/ForgeryLocalization/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Check for corrupted videos:

In [2]:
# corrupted_files = check_videos_for_corruption("AVDeepfake1Mpp/data/", max_workers=11)
# print(corrupted_files)

Goal: Run training for BA-TFD on a small subset of AV-Deepfake1M-PlusPlus to test the setup.

Args:

In [3]:
train_args = {
    "config": 'models/batfd/batfd.toml',
    "data_root": 'AVDeepfake1Mpp/data/',
    "batch_size": 8,
    "num_workers": 11,
    "gpus": 1,
    "precision": 32,
    "num_train": 100,
    "num_val": 100,
    "max_epochs": 2,
    "logger": "tensorboard",
    "resume": None
}

In [4]:
# prepare metadata
generate_metadata(train_args["data_root"])

Existing metadata file AVDeepfake1Mpp/data/train_metadata.json has been removed.
Metadata for train saved to AVDeepfake1Mpp/data/train_metadata.json
Existing metadata file AVDeepfake1Mpp/data/val_metadata.json has been removed.
Metadata for val saved to AVDeepfake1Mpp/data/val_metadata.json
Existing file list AVDeepfake1Mpp/data/testA_files.txt has been removed.
File list for testA saved to AVDeepfake1Mpp/data/testA_files.txt


In [5]:
#train(train_args)

Run inference on validation set, as no labels are provided for the AVDeepfake1M test set:

In [6]:
infer_args = {
    "output_dir": "predictions/",
    "config": train_args["config"],
    "checkpoint": "ckpt/batfd/20250707-184844/last.ckpt",  # update with actual checkpoint path
    "data_root": train_args["data_root"],
    "num_workers": train_args["num_workers"],
    "subset": "val",  # use val for 'testing'
    "gpus": train_args["gpus"]
}

In [7]:
infer(infer_args)

Using device: cuda:0


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Loaded model!
Loaded DataModule!
Load 2607 data in train.
Load 734 data in val.
Load 1419 data in testA.
Predicting DataLoader 0: 100%|██████████| 734/734 [01:28<00:00,  8.27it/s]


100%|██████████| 734/734 [00:02<00:00, 291.40it/s]


Inference complete. Results saved to predictions/batfd/20250708-210705/last_val.json


Evaluate the model: